In [1]:
import pandas as pd
file_path = r"C:\Users\141823\OneDrive - Etex Group\Desktop\Etex AU - Finance - Finance\19.Business Systems\Metal Report\BW Original Data.xlsx"
mapping_path=r"C:\Users\141823\OneDrive - Etex Group\Desktop\Etex AU - Finance - Finance\19.Business Systems\Metal Report\Mapping Template.xlsx"
# Load Excel file
xls = pd.ExcelFile(file_path)
xls1= pd.ExcelFile(mapping_path)

# Read "Raw Data" and "Region Mapping" sheets
df23=pd.read_excel(xls, sheet_name="2023", header=0)
df24=pd.read_excel(xls, sheet_name="2024", header=0)
df25=pd.read_excel(xls, sheet_name="2025", header=0)

#read mapping data
sales_office_mapping=pd.read_excel(xls1, sheet_name="Sales Office", header=0)
region_mapping=pd.read_excel(xls1, sheet_name="Region Mapping", header=0)
group_mapping=pd.read_excel(xls1, sheet_name="Grouping", header=0)


# Combine df23, df24, and df25
df = pd.concat([df23, df24, df25], ignore_index=True)



In [2]:
#-------------------------------------------Product Masterdata----------------------------
import pandas as pd
from openpyxl import load_workbook



# Read both material and customer master data
material_df = pd.read_excel(r"C:\Users\141823\OneDrive - Etex Group\Desktop\Etex AU - Finance - Finance\00. Master data\Material Master Data _Etex AU 5500.xlsx", sheet_name=0, header=None)
customer_df = pd.read_excel(r"C:\Users\141823\OneDrive - Etex Group\Desktop\Etex AU - Finance - Finance\00. Master data\Customer Master Data _Etex.XLSX", sheet_name='Customer Masterdata')

# Set the second row (index 1) as the column headers
material_df.columns = material_df.iloc[1]  # Set second row as header
material_df = material_df.iloc[2:].reset_index(drop=True)  # Drop first two rows

# Pick needed columns and remove duplicates
material_subset = material_df[['Material', 'Board / Metal','P&L Classification','Group 1 Category','BRAND - Siniat / Promat / Innova / Equitone','Profile Text','Item Group']].rename(
    columns={'BRAND - Siniat / Promat / Innova / Equitone': 'Brand',
             'P&L Classification':'P&L Category',
             'Group 1 Category':'Manual Classification'})
# material_subset = material_subset.drop_duplicates(subset=['Material', 'Brand'])

# Merge with Raw Data
df = df.merge(material_subset, on='Material', how='left')

# Filter only for 'SINIAT' and 'GTEK' brands
df = df[df['Brand'].isin(['SINIAT', 'GTEK'])]

#Filter only for 'Metal'
df = df[df['Board / Metal'] == 'METAL']


In [3]:
#------------------------------------Customer Masterdata-------------------------------
# Select columns by index
customer_subset = customer_df.iloc[:, [0, 4, 9]]

# Rename columns
customer_subset.columns = ['Customer','Sales Office', 'Customer Name Master']
customer_subset=customer_subset.drop_duplicates(subset=['Customer', 'Customer Name Master'])

#Merge with df
df=df.merge(customer_subset, on='Customer', how='left')

# Replace 'Customer Name' in df with 'Customer Name Master' and rename it back to 'Customer Name'
df['Customer Name'] = df['Customer Name Master']

# Drop the extra 'Customer Name Master' column
df = df.drop(columns=['Customer Name Master'])

#Create 'Customer Group',in Customer Name, group names that contain 'BUNNINGS' to 'BUNNINGS GROUP LTD' 

df['Customer Group'] = df['Customer Name'].apply(
    lambda x: 'BUNNINGS GROUP LTD' if 'BUNNINGS' in str(x).upper() else x
)


In [4]:
#--------------------------------Sales Office-------------
#Merge with Region Mapping to replace "Region" with "BW Region"
df = df.merge(sales_office_mapping, on='Sales Office', how='left')

#Select needed columns from region_mapping
region_mapping=region_mapping[['Region','BW Region']]

df=df.merge(region_mapping, on='Region', how='left')

# Replace "Region" with "BW Region", keeping original values if no match
df['Region'] = df['Sales Office Final'].fillna(df['BW Region'])

#Drop BW Region now, drop 'Brand' and 'Board / Metal'
df = df.drop(columns=['Sales Office Final','Brand','Board / Metal', 'BW Region'])

# build gross sales exclude Freight
df['Gross Sales(Exc Freight)']=(df['Gross Sales (Invoice)'].fillna(0)-df['Transport Surcharge'].fillna(0))

# Calculate Margin
df['Net Margin'] = (
    df['Actual Cost Of Goods Sold'].fillna(0) +
    df['Gross Sales(Exc Freight)'].fillna(0) +
    df['Provisions'].fillna(0)
)

# Calculate Margin after Transport
df['Net Margin after Transport'] = (
    df['Net Margin'] +
    df['Transport Surcharge'].fillna(0) +
    df['Transport to Customers'].fillna(0) +
    df['Interplant Transport'].fillna(0)
)

In [5]:
#----------------Group Mapping-------------------------

#Select needed columns from group_mapping
group_mapping=group_mapping[['Material', 'Gauge','Product Group']]

#drop duplicate in case group_mapping is duplicated
group_mapping=group_mapping.drop_duplicates(subset=['Material', 'Gauge','Product Group'])

#Merge with data frame now

df=df.merge(group_mapping, on='Material', how='left')

# Change all 'NT' to 'SA' as NT is part of SA budget
df.loc[df['Region'] == 'NT', 'Region'] = 'SA'


#-----------Profile text grouping
# List of special Profile Texts that should map to 'Others'
special_profiles = ['Screws','Armstrong','Façade System','Jamb Studs','Metal Anchor',
                    'Access Hatches','Reveal Angles','Acoustic Stud','Bought in Products for Resale',
                    'No Discount','Hardies Board','WBT Range','WBT - TAPEPRO','Exposed Grids']

# Create 'Profile Group'
df['Profile Group'] = df['Profile Text'].apply(
    lambda x: 'Others' if str(x).strip() in special_profiles else str(x).strip()
)



In [6]:
#-----------------------Create a function to add keys-----------------
def add_date_keys(df):
    # Extract Period and Year
    df[['Period', 'Year']] = df['Fiscal year/period'].str.extract(r'K4/(\d{3})\.(\d{4})')
    df['Year'] = df['Year'].astype(int)
    df['Period'] = df['Period'].astype(int)

    # Map month
    month_map = {
        1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr',
        5: 'May', 6: 'Jun', 7: 'Jul', 8: 'Aug',
        9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'
    }
    df['Month'] = df['Period'].map(month_map)

    # Create the Key
    df['Key'] = (
        df['Region'].astype(str) + '-' +
        df['Year'].astype(str) + '-' +
        df['Month'].astype(str) + '-' +
        df['Material'].astype(str)
    )
    return df


In [7]:
#------------------------------Budgeting-----------------

nsw=pd.read_excel(xls1, sheet_name="NSW Budget", header=0)
qld=pd.read_excel(xls1, sheet_name="QLD Budget", header=0)
vic=pd.read_excel(xls1, sheet_name="VIC Budget", header=0)
sa=pd.read_excel(xls1, sheet_name="SA Budget", header=0)
wa=pd.read_excel(xls1, sheet_name="WA Budget", header=0)

#Combine all budget data frame
budget_df= pd.concat([nsw,qld,vic,sa,wa], ignore_index=True)

#create a key for mapping with preliminary df
budget_df['Key'] = budget_df['Region'].astype(str) + '-' + budget_df['Year'].astype(str)+ '-' + budget_df['Month'].astype(str) + '-' + budget_df['Material'].astype(str)
budget_df=budget_df[['Key','Year','Month','Material','Region','Budget Unit','Budget $']]



In [8]:
# -------------------- Step 1: Select Margin Data --------------------
df1 = df[[
    'Fiscal year/period','Material', 'Material Description', 'Region',
    'Actual Cost Of Goods Sold', 'Quantity', 'Gross Sales (Invoice)', 'Gross Sales(Exc Freight)',
    'Transport Surcharge', 'Provisions', 'Transport to Customers', 'Interplant Transport',
    'P&L Category', 'Manual Classification', 'Profile Text', 'Item Group',
    'Net Margin', 'Net Margin after Transport', 'Gauge', 'Product Group','Profile Group'
]]

# Extract dimension info from df to enrich budget_df
df2 = df[[
    'Material', 'Material Description', 'P&L Category', 'Manual Classification', 'Profile Text', 'Item Group',
    'Gauge', 'Product Group','Profile Group'
]]

# -------------------- Step 2: Merge df2 into budget_df on Material --------------------
budget_df = budget_df.merge(df2.drop_duplicates('Material'), on='Material', how='left')

# -------------------- Step 3: Rename overlapping columns in budget_df --------------------
cols_to_fill = [
   'Material', 'Material Description', 'Region', 'P&L Category', 'Manual Classification',
    'Profile Text', 'Item Group', 'Gauge', 'Product Group', 'Year','Month','Profile Group'
]
budget_df = budget_df.rename(columns={col: f"{col}_budget" for col in cols_to_fill})

# -------------------- Step 4: Convert metrics to numeric in df and df1 --------------------
cols_to_convert = [
    'Actual Cost Of Goods Sold', 'Quantity', 'Gross Sales (Invoice)', 'Gross Sales(Exc Freight)',
    'Transport Surcharge', 'Provisions', 'Transport to Customers', 'Interplant Transport',
    'Net Margin', 'Net Margin after Transport'
]

df[cols_to_convert] = df[cols_to_convert].apply(pd.to_numeric, errors='coerce')
df1[cols_to_convert] = df1[cols_to_convert].apply(pd.to_numeric, errors='coerce')

# -------------------- Step 5: Group margin data --------------------
df1 = df1.groupby([
    'Fiscal year/period','Material', 'Material Description', 'Region',
    'P&L Category', 'Manual Classification', 'Profile Text', 'Item Group',
    'Gauge', 'Product Group','Profile Group'
], as_index=False).agg({
    'Actual Cost Of Goods Sold': 'sum',
    'Quantity': 'sum',
    'Gross Sales (Invoice)': 'sum',
    'Gross Sales(Exc Freight)': 'sum',
    'Transport Surcharge': 'sum',
    'Provisions': 'sum',
    'Transport to Customers': 'sum',
    'Interplant Transport': 'sum',
    'Net Margin': 'sum',
    'Net Margin after Transport': 'sum'
})

# -------------------- Step 6: Add primary keys --------------------
df = add_date_keys(df)
df1 = add_date_keys(df1)

# Standardize keys before merging
df1['Key'] = df1['Key'].astype(str).str.strip().str.upper()
budget_df['Key'] = budget_df['Key'].astype(str).str.strip().str.upper()

#df1.to_excel(r"C:\Users\141823\OneDrive - Etex Group\Desktop\df1.xlsx", index=False)
#budget_df.to_excel(r"C:\Users\141823\OneDrive - Etex Group\Desktop\Budget.xlsx", index=False)

C:\Users\141823\AppData\Local\Temp\ipykernel_27868\1336852449.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[cols_to_convert] = df1[cols_to_convert].apply(pd.to_numeric, errors='coerce')


In [9]:

# -------------------- Step 7: Outer merge margin with budget --------------------
df1 = df1.merge(budget_df, on='Key', how='outer')

# -------------------- Step 8: Fill missing values in df1 using budget fields --------------------
for col in cols_to_fill:
    df1[col] = df1[col].fillna(df1[f"{col}_budget"])

# Drop the temporary _budget columns
df1.drop(columns=[f"{col}_budget" for col in cols_to_fill], inplace=True)

# -------------------- Step 9: Fill nulls in metric columns with 0 --------------------
df1[cols_to_convert] = df1[cols_to_convert].fillna(0)

# -------------------- Step 10: Budget columns formatting --------------------
df1[['Budget Unit', 'Budget $']] = df1[['Budget Unit', 'Budget $']].fillna(0)
df1['Budget Unit'] = df1['Budget Unit'].astype(float).round(2)
df1['Budget $'] = df1['Budget $'].astype(float).round(2)

#----------------------------------------------------Data Cleaning---------------------------------
#------------------------------Drop fiscal year and period
df1 = df1.drop(columns=['Fiscal year/period', 'Period'])

# drop row with 0 data for both margin and budget
cols_check = ['Net Margin', 'Net Margin after Transport', 'Budget Unit', 'Budget $']
# Drop rows where ALL four columns are 0 in Budget file
df1 = df1[~(df1[cols_check].fillna(0) == 0).all(axis=1)]

#do the same for margin report
cols_check1=['Net Margin', 'Net Margin after Transport', 'Gross Sales(Exc Freight)']
df = df[~(df[cols_check1].fillna(0) == 0).all(axis=1)]

In [10]:
# ------------------- Create MoM and YoY columns for Margin report

# Step 1: Sort by the lowest grain
df = df.sort_values(['Customer', 'Material', 'Year', 'Period'])

# Step 2: Calculate Previous Period Revenue (for MoM)
df['Prev_Month_Revenue'] = df.groupby(['Customer', 'Material'])['Gross Sales(Exc Freight)'].shift(1)

# Step 3: Calculate MoM Variance
df['MoM_Variance'] = df['Gross Sales(Exc Freight)'] - df['Prev_Month_Revenue']

# Step 4: Prepare for YoY Join
df_yoy = df[['Year', 'Period', 'Customer', 'Material', 'Gross Sales(Exc Freight)']].copy()
df_yoy['Year'] = df_yoy['Year'] + 1  # shift to next year
df_yoy = df_yoy.rename(columns={'Gross Sales(Exc Freight)': 'Prev_Year_Revenue'})

# Step 5: Merge YoY data
df = df.merge(df_yoy, on=['Year', 'Period', 'Customer', 'Material'], how='left')

# Step 6: Calculate YoY Variance
df['YoY_Variance'] = df['Gross Sales(Exc Freight)'] - df['Prev_Year_Revenue']

df['Prev_Month_Revenue']=df['Prev_Month_Revenue'] .fillna(0)
df['MoM_Variance'] =df['MoM_Variance'] .fillna(0)
df['Prev_Year_Revenue']=df['Prev_Year_Revenue'].fillna(0)
df['YoY_Variance']=df['YoY_Variance'].fillna(0)



In [11]:
#--------------------Save df to final file------------------
# Load the workbook
file_path=r"C:\Users\141823\OneDrive - Etex Group\Desktop\Etex AU - Finance - Finance\19.Business Systems\Metal Report\Metal Report-Raw Data.xlsx"
# Load workbook once
wb = load_workbook(file_path)

# Define a reusable write function
def write_dataframe_to_sheet(wb, sheet_name, df_to_write):
    ws = wb[sheet_name]
    
    # Clear old data except headers
    ws.delete_rows(2, ws.max_row)

    # Write headers
    for c_idx, column in enumerate(df_to_write.columns, start=1):
        ws.cell(row=1, column=c_idx, value=column)

    # Write data rows
    for r_idx, row in enumerate(df_to_write.itertuples(index=False), start=2):
        for c_idx, value in enumerate(row, start=1):
            ws.cell(row=r_idx, column=c_idx, value=value)

    # Auto-fit column widths
    for col in ws.columns:
        max_length = 0
        col_letter = col[0].column_letter
        for cell in col:
            try:
                if cell.value:
                    max_length = max(max_length, len(str(cell.value)))
            except:
                pass
        ws.column_dimensions[col_letter].width = max_length + 2

# Call function for each DataFrame
write_dataframe_to_sheet(wb, "Margin Data", df)
write_dataframe_to_sheet(wb, "Budget Data", df1)

# Save workbook
wb.save(file_path)